In [ ]:
from datetime import datetime, timedelta
import mysql.connector
import weaviate
import tiktoken
import requests
import openai
import json


openai.api_key = '<<Your API Key>>'
auth_config = weaviate.auth.AuthApiKey(api_key="<<Your API Key>>")
client = weaviate.Client(
    url = "Replace with your endpoint",  
    auth_client_secret=auth_config,
    additional_headers = {
        "X-Cohere-Api-Key" : "<<Your API Key>>" # Replace with your API key
    }
)

In [ ]:
import cohere
openai.api_key = '<<Your API Key>>'
co = cohere.Client('<<Your API Key>>')

In [ ]:
def insert_data(mydb, vec_class, uuid, question, chat_conv):
    # Check if UUID already exists in the database
    mycursor = mydb.cursor()
    sql_select_query = "SELECT * FROM chat_history WHERE uuid = %s AND vec_class = %s"
    mycursor.execute(sql_select_query, (uuid, vec_class))
    result = mycursor.fetchone()
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
    if result: # UUID already exists, update the existing row
        sql_update_query = "UPDATE chat_history SET prev_queries = CONCAT(prev_queries, CONCAT('###', %s)), chat_history = CONCAT(chat_history, CONCAT('###', %s)), logged_at = NOW() WHERE uuid = %s AND vec_class = %s"
        mycursor.execute(sql_update_query, (question, chat_conv, uuid, vec_class))
        mydb.commit()
    else: # UUID does not exist, insert a new row
        sql_insert_query = "INSERT INTO chat_history (vec_class, uuid, prev_queries, chat_history, logged_at) VALUES (%s, %s, %s, %s, %s)"
        mycursor.execute(sql_insert_query, (vec_class, uuid, question, chat_conv, timestamp))
        mydb.commit()
    

# Define function for retrieving the last conversation for a given vec_class
def get_last_conversation(mydb, uuid, vec_class):
    
    mycursor = mydb.cursor()
    sql = "SELECT chat_history FROM chat_history WHERE uuid = %s AND vec_class = %s"
    val = (uuid, vec_class)
    mycursor.execute(sql, val)
    result = mycursor.fetchone() #result[0] stores the last conversations ; if result is not none: return result[0]; similar in get_prev_queries
    
    
    if result is not None:
        conversations = result[0].split("###")
        context = "###".join(conversations[-5:])  # Keep only the most recent 3 conversations
        prev_responses = "###".join([context] + conversations[:-3]) 
        return context
    else:
        return None

# Define function for retrieving the last conversation for a given vec_class
def get_prev_queries(mydb, uuid, vec_class):
    
    mycursor = mydb.cursor()
    sql = "SELECT prev_queries FROM chat_history WHERE uuid = %s AND vec_class = %s"
    val = (uuid, vec_class)
    mycursor.execute(sql, val)
    result = mycursor.fetchone()
     
    
    if result is not None:
        conversations = result[0].split("###")
        context = "###".join(conversations[-5:])  # Keep only the most recent 3 conversations
        prev_queries = "###".join([context] + conversations[:-3])  
        return context
    else:
        return None

In [ ]:
def create_context(question, last_conversation, vec_class):
    """
    Find most relevant context for a question via Pinecone search
    """
    if vec_class.upper() == 'QUESTION':
        vec_class = 'Question'
    elif vec_class.upper() == 'PERSONAL':
        vec_class = 'Personal'
    elif vec_class.upper() == 'DOCUMENTS':
        vec_class = 'Documents'
        
    if last_conversation == None:
        last_conversation = ""    
    result = (
        client.query
        .get(vec_class, ["question", "answer"])
        .with_hybrid(question+last_conversation, alpha=0.5)
        .with_limit(10)
        .do()
    )['data']['Get'][vec_class]
    results = []
    for res in result:
        results.append(res['answer'])
    reranked_results = co.rerank(query=question, documents=results, top_n=3, model="rerank-multilingual-v2.0")
    print("Reranked Results: ", reranked_results)
    
    paragraph = ''
    for q in result:
        paragraph += q['question'] + ' ' + q['answer']
    # concatenated_answers = ' '.join(answers)
    # return concatenated_answers
    return paragraph

In [ ]:
def generate_response(
    prompt,
    debug=False,
    max_tokens=100,
    stop_sequence=None,
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    try:
        #print(instruction.format(context, question))
        response = openai.Completion.create(
            model = "text-davinci-003",
            prompt=prompt,
            temperature=0.5,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=True,
            best_of = 
        )

        generated_text = ""
        for line in response:
            generated_text += line.choices[0].text  # extract the text from the StreamingText object
        
        return generated_text.split("###")[0].strip()

    except Exception as e:
        print(e)
        return ""

In [ ]:

def calculate_tokens(prompt):
    encoding = tiktoken.encoding_for_model("text-davinci-003")
    num_tokens = len(encoding.encode(prompt))
    return num_tokens

In [ ]:
def chatbot(uuid, vec_class, question, debug=False):
    # Initialize Database
    mydb = mysql.connector.connect(
        host="<<Database endpoint>>",
        user="",
        password="",
        database=""
    )

    # Retrieve last conversation for the vec_class
    last_conversation = get_last_conversation(mydb, uuid, vec_class)
    prev_queries = get_prev_queries(mydb, uuid, vec_class)

    # Get Context for answering question
    context = create_context(
        question,
        prev_queries,
        vec_class  
    )

    # Print context while debugging

        
    
    # If there was a previous conversation, include it in the prompt
    if last_conversation is not None:
        prompt = f'''You are a search and answer bot. Use the context and the Previous Responses to return only the answer relevant to the {question}. Do not repeat Previous Responses.
            If relevant link is available in the context make sure to include them in the answer.
            and if the question can't be answered based on the context,
            say \"Sorry! I have no information available for that at the moment\"\n\n Context: \n{context}\n\n---\n\n Previous Responses: {last_conversation}\n Question: {question}\n Answer:
        '''
    else:
        prompt = f'''You are a search and answer bot. Use the context to return only the answer relevant to the {question}.
            If relevant link is available in the context include them in the answer.
            and if the question can't be answered based on the context,
            say \"Sorry! I have no information available for that at the moment\"\n\n Context: \n{context}\n\n---\n\n Question: {question}\n Answer:
        '''
    
    num_tokens = calculate_tokens(prompt)

    if debug:
        print("Last 3 Queries: ", prev_queries)
        print("Last 3 Conversation: ", last_conversation)
        print(context)
        print("Num Tokens: ", num_tokens)
    
    # Generate response from OpenAI API
    response = generate_response(prompt)
    
    # Insert conversation into the database
    # uuid = str(uuid.uuid4())
    # chat_conv = f"{context}\n\n---\n\nQuestion: {question}\nAnswer: {response}"
    chat_conv = f"Question: {question} Answer: {response}"
    insert_data(mydb, vec_class, uuid, question, chat_conv)

    mydb.close()
    
    # Return response to the user
    return response

In [ ]:
query =''
uuid=''
vec_class = ''
lang_code = detect_lang(query)
print(lang_code)
if lang_code == 'EN':
    result = chatbot(uuid, vec_class, query, debug = True)
else:
    trans_query = user_lang(query, 'EN')
    answer = chatbot(uuid, vec_class, trans_query, debug = True)
    result = user_lang(answer, lang_code)

print(result)

